In [ ]:
import monai
import torch
from monai.optimizers import WarmupCosineSchedule
import yaml
import matplotlib.pyplot as plt
import numpy as np

with open('/home/benito/script/NephroBIT/KPIs24/config_train_Unet.yaml', 'r') as file:
    cfg = yaml.safe_load(file)

print(cfg)

device = torch.device(f"cuda:{cfg['device_number']}" if torch.cuda.is_available() else "cpu")

model = monai.networks.nets.UNet(
        spatial_dims=cfg['model']['params']['spatial_dims'],
        in_channels=cfg['model']['params']['in_channels'],
        out_channels=cfg['model']['params']['out_channels'],
        channels=cfg['model']['params']['f_maps_channels'],
        strides=cfg['model']['params']['strides'],
        num_res_units=cfg['model']['params']['num_res_units'],
    ).to(device)

warmup_steps= int(cfg['model']['scheduler']['params']['warmup_epochs'] * 4139 / cfg['training']['train_batch_size'])
t_total = int(cfg['training']['epoch']* 4139 / cfg['training']['train_batch_size'])

optimizer = torch.optim.Adam(model.parameters(), cfg['model']['optimizer']['params']['learning_rate'])
scheduler = WarmupCosineSchedule(optimizer, warmup_steps=warmup_steps, t_total=t_total, cycles = cfg['model']['scheduler']['params']['cycles'], end_lr=1e-9)
lr_values = []
for steps in range(t_total):
    lr = scheduler.get_lr()[0]
    lr_values.append(lr)
    optimizer.step()
    scheduler.step()
plt.plot(range(t_total), lr_values, label=f'Cycle {cfg["model"]["scheduler"]["params"]["cycles"]}')
plt.legend()
plt.show()

In [ ]:
for cycle in np.linspace(0, 5, 11):
    optimizer = torch.optim.Adam(model.parameters(), cfg['model']['optimizer']['params']['learning_rate'])
    scheduler = WarmupCosineSchedule(optimizer, warmup_steps=warmup_steps, t_total=t_total, cycles = cycle+1, end_lr=1e-9)
    lr_values = []
    for steps in range(t_total):
        lr = scheduler.get_lr()[0]
        lr_values.append(lr)
        optimizer.step()
        scheduler.step()
    plt.plot(range(t_total), lr_values, label=f'Cycle {cycle+1}')
plt.legend()
plt.show()
    

In [ ]:
warmup_steps
